 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 2: Optimisation sans contraintes
Tangi Migot

In [6]:
using Pkg
Pkg.add("ADNLPModels")
Pkg.add("NLPModels")
using ADNLPModels, LinearAlgebra, NLPModels, Printf

    Updating registry at `C:\Users\Jean-Luc\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Jean-Luc\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Jean-Luc\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Jean-Luc\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Jean-Luc\.julia\environments\v1.7\Manifest.toml`


On pourra trouver de la documentation sur `ADNLPModels` et `NLPModels` ici:
- [juliasmoothoptimizers.github.io/NLPModels.jl/dev/](https://juliasmoothoptimizers.github.io/NLPModels.jl/dev/)
- [juliasmoothoptimizers.github.io/ADNLPModels.jl/dev/](https://juliasmoothoptimizers.github.io/ADNLPModels.jl/dev/)

### Exercice 1: Newton avec recherche linéaire - amélioration du code

Ci-dessous, vous avez le code de deux fonctions qui ont été vues dans les capsules, la recherche linéaire qui satisfait Armijo, et une méthode de Newton avec cette recherche linéaire. Le but de ce laboratoire est d'implémenter d'autres méthodes utiles pour résoudre des problèmes de grandes dimensions.

In [1]:
#Amélioration possibles: return also the value of f
function armijo(xk, dk, fk, gk, f)
  slope = dot(gk, dk) #doit être <0
  t = 1.0
  while f(xk + t * dk) > fk + 1.0e-4 * t * slope
    t /= 1.5
  end
  return t
end

armijo (generic function with 1 method)

In [7]:
#Test pour vérifier que la fonction armijo fonctionne correctement.
using Test #le package Test définit (entre autre) la macro @test qui permet de faire des tests unitaires :-)
xk = ones(2)
gk = g(xk)
dk = - gk
fk = f(xk)
t  = armijo(xk, dk, fk, gk, f)
@test t < 1
@test f(xk + t * dk) <= fk + 1.0e-4 * t * dot(gk,dk)

xk = [1.5, 0.5]
fk = f(xk)
gk = g(xk)
dk = - gk
t  = armijo(xk, dk, fk, gk, f)
@test t < 1
@test f(xk + t * dk) <= f(xk) + 1.0e-4 * t * dot(g(xk),dk)


Test Passed
  Expression: f(xk + t * dk) <= f(xk) + 0.0001 * t * dot(g(xk), dk)
   Evaluated: -0.37311385459533614 <= -0.00017777777777777779

In [8]:
function newton_armijo(f, g, H, x0)
  xk  = x0
  fk  = f(xk)
  gk = g(xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100
    Hk = H(xk)
    dk = - Hk \ gk
    slope = dot(dk, gk)
    λ = 0.0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
    end
    t = armijo(xk, dk, fk, gk, f)
    xk += t * dk
    fk = f(xk)
    gk = g(xk)
    gnorm = norm(gk)
    k += 1
    @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end

newton_armijo (generic function with 1 method)

In [9]:
# Problème test:
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1) # fonction objectif vue en classe
g(x) = 6 * [x[1]^2 - x[1] - 2*x[1]*x[2] + x[2]^2 + x[2]; -x[1]^2 + 2*x[1]*x[2] + x[1]] # le gradient de f
H(x) = 6 * [2*x[1]-1-2*x[2] -2*x[1]+2*x[2]+1; -2*x[1]+2*x[2]+1 2*x[1]] # la Hessienne de f

H (generic function with 1 method)

In [19]:
sol = newton_armijo(f, g, H, [.5, .5])
@test g(sol) ≈ zeros(2) atol = 1.0e-6
@printf "%s" sol

 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00
 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 
 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00
 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 
[2.3230573665069826e-8, 2.3230573665069826e-8]

On veut améliorer le code de la fonction `newton_armijo` avec les ajouts suivants:
- Changer les paramètre d'entrées de la fonction pour un `nlp`
- Avant d'appeler la recherche linéaire, si `slope = dot(dk, gk)` est plus grand que `-1.0e-4 * norm(dk) * gnorm`, on modifie le système. On fait maximum 5 mise à jour de `λ`, sinon on prend l'opposé du gradient.
```
    λ = 0.0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
    end
```
Ajouter un compteur sur le nombre de mises à jour de `λ` et ajuster `dk = - gk` si la limite est atteinte.
- On veut aussi détecter et éventuellement arrêter la boucle `while` si la fonction objectif `fk` devient trop petite/négative, i.e. le problème est non-bornée inférieurement.
- On veut ajouter deux critères d'arrêts supplémentaires: 
  - un compteur sur le nombre d'évaluations de f (maximum 1000). Utiliser `neval_obj(nlp)`.
  -  une limite de temps d'execution. Utiliser la fonction `time()`.
- Enfin, on voudrait aussi voir un message à l'écran si l'algorithme n'a pas trouvé la solution, i.e. il s'est arrêté à cause de la limite sur le nombre d'itérations, temps, évaluation de fonctions, problème non-borné.

In [14]:
#SOLUTION: fonction à modifier
function newton_armijo(nlp, x0)
  xk  = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  start = time()
  @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && neval_obj(nlp) < 1000 && time()-start <= 60.0 && abs(obj(nlp, xk)) <= 1.0e8
    Hk = hess(nlp, xk)
    dk = - Hk \ gk
    slope = dot(dk, gk)
    λ = 0.0
    lcounter = 1
    while slope ≥ -1.0e-4 * norm(dk) * gnorm && lcounter <= 7
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
      lcounter += 1
    end
    if lcounter > 7
      dk = - gk
      @printf "%d Iterations of lambda reached" "lcounter"
    end
    t = armijo(xk, dk, fk, gk, f)
    xk += t * dk
    fk  = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end

 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00


newton_armijo (generic function with 2 methods)

In [15]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
x0 =  [.5, .5]
nlp = ADNLPModel(f, x0)

newton_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00
 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 


2-element Vector{Float64}:
 2.323057366509693e-8
 2.323057366509693e-8

In [ ]:
# Tests secrets 1: contactez votre professeur pour avoir ces tests additionels si vous êtes prêt.

### Exercice 2: LDLt-Newton avec recherche linéaire

On va maintenant modifier la méthode de Newton vu précédemment pour utiliser un package qui s'occupe de calculer une factorisation de la matrice hessienne tel que:
$$
\nabla^2 f(x) = LDL^T.
$$
Ce type de factorisation n'est possible que si la matrice hessienne est définie positive, dans le cas contraire on a besoin de régularisé le système comme dans l'exercice précédent.

Pour résoudre le système linéaire en utilisant cette factorisation, on va utiliser le package [`LDLFactorizations`](https://github.com/JuliaSmoothOptimizers/LDLFactorizations.jl):

In [20]:
using Pkg
Pkg.add("LDLFactorizations")
using LDLFactorizations, LinearAlgebra

   Resolving package versions...
  No Changes to `C:\Users\Jean-Luc\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Jean-Luc\.julia\environments\v1.7\Manifest.toml`


Un tutoriel sur l'utilisation de `LDLFactorizations` est disponible sur la documentation du package sur github ou encore [à ce lien](https://juliasmoothoptimizers.github.io/LDLFactorizations.jl/dev/tutorial/).

Voici un exemple d'utilisation de ce package. La matrice dont on veut calculer la factorisation doit être de type `Symmetric`.

In [21]:
A = ones(2,2) #cette matrice symétrique, mais pas du type Symmetric
              #à noter que cette matrice n'est pas définie positive.
typeof(A) <: Symmetric #false
A = Symmetric(A)
typeof(A) <: Symmetric #true :)

true

Deuxième étape, le package fait une phase d'analyse de la matrice avec `ldl_analyze` en créant une structure pratique pour les diverses fonctions du package.

In [22]:
A = -rand(2, 2)
sol = rand(2)
b = A*sol #on veut résoudre le système A*x=b

# LDLFactorizations va en réalité demander la matrice triangulaire supérieure
A = Symmetric(triu(A), :U)

S = ldl_analyze(A)

ldl_factorize!(A, S) 
x = S \ b # x = A \b ça va être résolu par Julia 
norm(A * x - b)

0.0

La matrice `A` factorisée par $LDL^T$ n'était pas forcément définie positive. On peut le voir sur les valeurs de $D$.

In [4]:
S.d #c'est le vecteur qui correspond à la matrice diagonale D.

2-element Vector{Float64}:
 -0.3619689267912928
  0.11895679244576385

Pour l'optimisation, dans le cas où des valeurs de $D$ sont négatives, i.e. `minimum(S.d) <= 0.`, on ajoutera une correction pour être sûr d'obtenir une direction de descente. On pourra choisir un des deux:
- `S.d   = abs.(S.d)` #la valeur absolue des valeurs négatives
- `S.d .+= minimum(S.d) + epsilon`

`LDLFactorizations` a une façon automatique d'utiliser la deuxième option, il suffit de modifier la structure `S` obtenue par `ldl_analyze` avant la factorisation:
```
S.tol = 1e-8 #régularise les valeurs plus petites que cette tolérance
S.n_d = 0 #taille de la régularisation
S.r1 = -eps()^(1/4)
S.r2 = eps()^(1/2)
```

#### Utiliser cette technique pour calculer la direction de descente:

In [23]:
# Solution: copier-coller votre newton_armijo ici et modifier le calcul de la direction avec LDLFactorizations
function newton_ldlt_armijo(nlp, x0)
  xk  = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  start = time()
  @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && neval_obj(nlp) < 1000 && time()-start <= 60.0 && abs(obj(nlp, xk)) <= 1.0e8
    Hk = hess(nlp, xk)
    
    Hk = Symmetric(triu(Hk), :U)
    S = ldl_analyze(Hk)
    S.tol = 1e-8 #régularise les valeurs plus petites que cette tolérance
    S.n_d = 0 #taille de la régularisation
    S.r1 = -eps()^(1/4)
    S.r2 = eps()^(1/2)
    ldl_factorize!(Hk, S)
    dk = -S \ gk

    t = armijo(xk, dk, fk, gk, f)
    xk += t * dk
    fk  = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end


newton_ldlt_armijo (generic function with 1 method)

In [24]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, [.5, .5])

newton_ldlt_armijo(nlp, x0)

UndefVarError: UndefVarError: ldl_analyse not defined

In [ ]:
# Tests secrets 2: contactez votre professeur pour avoir ces tests additionels si vous êtes prêt.

### Exercice 3: Méthode quasi-Newton: BFGS

#### Méthode quasi-Newton: BFGS
Pour des problèmes de très grandes tailles, il est parfois très coûteux d'évaluer la hessienne du problème d'optimisation (et même le produit hessienne-vecteur). La famille des méthode *quasi-Newton* construit une approximation $B_k$ symétrique définie positive de la matrice Hessienne en utilisant seulement le gradient et en mesurant sa variation, et permet quand même d'améliorer significativement les performances comparé à la méthode du gradient.
$$
s_k = x_{k+1} - x_k, \quad y_k = \nabla f(x_{k+1}) - \nabla f(x_k).
$$
Par ailleurs la matrice $B_k$ est aussi construite de façon à ce que l'inverse soit connue, il n'y a donc pas de système linéaire à résoudre.

La méthode la plus connue dans la famille des méthodes quasi-Newton, est la méthode BFGS (Broyden - Fletcher, Goldfarb, and Shanno), et utilise la formule suivante pour calculer l'inverse de $B_k$ que l'on note $H_k$:
$$
H_{k+1} = (I - \rho_k s_ky_k^T)H_k(I-\rho_ky_ks_k^T) + \rho_ks_ks_k^T, \quad \rho_k = \frac{1}{y_k^Ts_k}.
$$
L'algorithme est presque le même que la méthode de Newton à la différence qu'il n'y a pas de système linéaire à résoudre et la direction $d_k$ est à coup sûr une direction de descente. Ainsi la direction de descente est calculée comme suit:
$$
d_k = - H_k \nabla f(x_k).
$$

Comment choisir la matrice $H_0$? On peut éventuellement choisir $I$. Une alternative est d'utiliser $H_0=I$ pour la première itération et ensuite mettre $H_0$ à jour avant de calculer $H_1$ en utilisant:
$$
H_0 = \frac{y_k^Ts_k}{y_k^Ty_k}I.
$$

**Important**: pour s'assurer que la matrice $H_k$ reste définie positive à toutes les itérations, il faut s'assurer que $y_k^Ts_k>0$. C'est toujours vrai pour des fonctions convexes, mais pas nécessairement dans le cas général. On pourra tester ici la version "skip" qui ne mets pas à jour quand cette condition n'est pas vérifiée.

In [ ]:
# Solution: copier-coller votre newton_armijo ici et modifier le calcul de la direction avec la méthode de BFGS inverse skip.
function bfgs_quasi_newton_armijo(nlp, x0)
  xk = x0
  #à compléter
  return xk
end

In [ ]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, zeros(2))

In [ ]:
# Tests secrets 3: contactez votre professeur pour avoir ces tests additionels si vous êtes prêt.

### Exercice 4: application à un problème de grande taille

On va ajouter le package `OptimizationProblems` qui contient, comme son nom l'indique, une collection de problème d'optimisation disponible au format de `JuMP` (dans le sous-module `OptimizationProblems.PureJuMP`) et de `ADNLPModel` (dans le sous-module `OptimizationProblems.ADNLPProblems`).

In [ ]:
using ADNLPModels, OptimizationProblems.ADNLPProblems # Attention si vous ne faites pas using ADNLPModels avant ça ne fonctionne pas!

n = 500
model = genrose(n=n)
@test typeof(model) == ADNLPModel{Float64, Vector{Float64}}

Si vous le souhaitez, il est possible d'accéder à certaines informations sur le problème en accédant à son meta:

In [ ]:
using OptimizationProblems
OptimizationProblems.genrose_meta

Résoudre le problème `genrose` et un autre problème de la collection en utilisant vos algorithmes précédents.
Avant d'utiliser l'algorithme on testera que le problème est bien sans contrainte avec:

In [ ]:
unconstrained(nlp) #qui retourne vrai si `nlp` est un problème sans contraintes.